This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/model_conversion/statsmodels).

## Introduction

Organizations can deploy a Machine Learning (ML) model based on the [statsmodels](https://www.statsmodels.org/stable/index.html) directly into Wallaroo through the following process.  This conversion process transforms the model into an open format that can be run across different frameworks at compiled C-language speeds.

This example provides the following:

* `train-statsmodel.ipynb`: A sample Jupyter Notebook that trains a sample model.  The model predicts how many bikes will be rented on each of the next 7 days, based on the previous 7 days' bike rentals, temperature, and wind speed.  Additional files to support this example are:
  * `day.csv`: Data used to train the sample `statsmodel` example.
  * `infer.py`: The inference script that is part of the `statsmodel`.
* `convert-statsmodel-tutorial.ipynb`: A sample Jupyter Notebook that demonstrates how to upload, convert, and deploy the `statsmodel` example into a Wallaroo instance.    Additional files to support this example are:
  * `bike_day_model.pkl`: A `statsmodel` ML model trained from the `train-statsmodel.ipynb` Notebook.

    **IMPORTANT NOTE:** The `statsmodel` ML model is composed of two parts that are contained in the .pkl file:

    * The pickled Python runtime expects a dictionary with two keys: `model` and `script`:

      * `model`—the pickled model, which will be automatically loaded into the python runtime with the name 'model'
      * `script`—the text of the python script to be run, in a format similar to the existing python script steps (i.e. defining a wallaroo_json method which operates on the data).  In this cae, the file `infer.py` is the script used.

  * `bike_day_eval.json`: Evaluation data used to test the model's performance.

## Steps

The following steps will perform the following:
    
1. Upload the `statsmodel` ML model `bike_day_model.pkl` into a Wallaroo.
2. Deploy the model into a pipeline.
3. Run a test inference.
4. Undeploy the pipeline.

### Import Libraries

The first step is to import the libraries that we will need.

In [22]:
import json
import os
import datetime

import wallaroo
from wallaroo.object import EntityNotFoundError

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Initialize connection

Start a connect to the Wallaroo instance and save the connection into the variable `wl`.

In [23]:
# Login through local Wallaroo instance

#wl = wallaroo.Client()

# SSO login through keycloak

wallarooPrefix = "YOUR PREFIX"
wallarooSuffix = "YOUR PREFIX"

wallarooPrefix = "sparkly-apple-3026"
wallarooSuffix = "wallaroo.community"

# wallarooPrefix = "squishy-wallaroo-6187"
# wallarooSuffix = "wallaroo.dev"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Arrow Support

As of the 2023.1 release, Wallaroo provides support for dataframe and Arrow for inference inputs.  This tutorial allows users to adjust their experience based on whether they have enabled Arrow support in their Wallaroo instance or not.

If Arrow support has been enabled, `arrowEnabled=True`. If disabled or you're not sure, set it to `arrowEnabled=False`

The examples below will be shown in an arrow enabled environment.

In [24]:
import os
arrowEnabled=True
os.environ["ARROW_ENABLED"]=f"{arrowEnabled}"


### Set Configurations

The following will set the workspace, model name, and pipeline that will be used for this example.  If the workspace or pipeline already exist, then they will assigned for use in this example.  If they do not exist, they will be created based on the names listed below.

In [25]:
workspace_name = 'bikedayevalworkspace'
pipeline_name = 'bikedayevalpipeline'
model_name = 'bikedaymodel'
model_file_name = 'bike_day_model.pkl'

## Set the Workspace and Pipeline

This sample code will create or use the existing workspace `bike-day-workspace` as the current workspace.

In [26]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)
pipeline

name,bikedayevalpipeline
created,2023-02-22 16:42:58.772535+00:00
last_updated,2023-02-22 16:57:04.536537+00:00
deployed,False
tags,
versions,"b778fd68-1cde-4083-9c78-d3d5fb3d3b99, b709a722-2a90-4583-98a2-5137d4116bc9, e7cb3a85-7c9f-4803-a98a-2dc18036079b"
steps,bikedaymodel


### Upload Pickled Package Statsmodel Model

Upload the statsmodel stored into the pickled package `bike_day_model.pkl`.  See the Notebook `train-statsmodel.ipynb` for more details on creating this package.

Note that this package is being specified as a `python` configuration.

In [27]:
file_name = "bike_day_model.pkl"

bike_day_model = wl.upload_model(model_name, model_file_name).configure(runtime="python")

### Deploy the Pipeline

We will now add the uploaded model as a step for the pipeline, then deploy it.

In [28]:
pipeline.add_model_step(bike_day_model)

name,bikedayevalpipeline
created,2023-02-22 16:42:58.772535+00:00
last_updated,2023-02-22 16:57:04.536537+00:00
deployed,False
tags,
versions,"b778fd68-1cde-4083-9c78-d3d5fb3d3b99, b709a722-2a90-4583-98a2-5137d4116bc9, e7cb3a85-7c9f-4803-a98a-2dc18036079b"
steps,bikedaymodel


In [29]:
pipeline.deploy()

name,bikedayevalpipeline
created,2023-02-22 16:42:58.772535+00:00
last_updated,2023-02-22 17:09:45.126166+00:00
deployed,True
tags,
versions,"c5faeb70-db31-4406-9d63-df46d024d44b, b778fd68-1cde-4083-9c78-d3d5fb3d3b99, b709a722-2a90-4583-98a2-5137d4116bc9, e7cb3a85-7c9f-4803-a98a-2dc18036079b"
steps,bikedaymodel


In [30]:
pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.48.1.42',
   'name': 'engine-6cbff4f8bf-m64s6',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'bikedayevalpipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'bikedaymodel',
      'version': 'cd87c9fe-a695-4148-8399-ac8f6311ee9c',
      'sha': '1bb486598732259efdd131b45d471e165b594d5443928ea9e50fa4c7e0b1b718',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.48.1.43',
   'name': 'engine-lb-74b4969486-8prh2',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

### Run Inference

Perform an inference from the evaluation data JSON file `bike_day_eval.json`.

In [31]:
if arrowEnabled is True:
    results = pipeline.infer_from_file('bike_day_eval.df.json')
else:
    results = pipeline.infer_from_file('bike_day_eval.json')
#display(results)

HTTPError: [Errno 500 Server Error: Internal Server Error for url: https://sparkly-apple-3026.api.wallaroo.community/v1/api/pipelines/infer/bikedayevalpipeline-39?dataset%5B%5D=%2A&dataset.exclude%5B%5D=metadata&dataset.separator=.] [{"error":"Error running inference <class 'KeyError'>, \"None of [Index(['temp', 'holiday', 'workingday', 'windspeed'], dtype='object')] are in the [columns]\"","error_type":"InternalError"}]

### Undeploy the Pipeline

Undeploy the pipeline and return the resources back to the Wallaroo instance.

In [ ]:
pipeline.undeploy()

name,bikedayevalpipeline
created,2023-02-22 16:42:58.772535+00:00
last_updated,2023-02-22 16:57:04.536537+00:00
deployed,False
tags,
versions,"b778fd68-1cde-4083-9c78-d3d5fb3d3b99, b709a722-2a90-4583-98a2-5137d4116bc9, e7cb3a85-7c9f-4803-a98a-2dc18036079b"
steps,bikedaymodel
